In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib qt

In [2]:
stem = "/Users/bandari/Documents/git.repos/gpi2_misc/notebooks_for_development/data/"

In [3]:
file_name = stem + "20220912_temperature.log"

In [4]:
df = pd.read_csv(file_name, skiprows=7, names=["day","time","molyblock","tita_det_head","intermed_opt_bench",
                                               "intermed_block_ccr1","cold_tip_ccr2",
                                               "intermed_block_ccr2","cold_tip_ccr1","asic_strap",
                                               "opt_bench","cold_shield","output_a","output_b"])

In [5]:
df

,day,time,molyblock,tita_det_head,intermed_opt_bench,intermed_block_ccr1,cold_tip_ccr2,intermed_block_ccr2,cold_tip_ccr1,asic_strap,opt_bench,cold_shield,output_a,output_b
0,22/09/12,11:43:12,291.294,291.727,292.334,291.292,295.575,292.808,291.547,291.631,292.189,291.958,0.0,0.0
1,22/09/12,11:44:12,291.294,291.727,292.334,291.292,295.576,292.807,291.547,291.631,292.189,291.958,0.0,0.0
2,22/09/12,11:45:12,291.294,291.727,292.334,291.292,295.576,292.808,291.547,291.631,292.189,291.958,0.0,0.0
3,22/09/12,11:46:12,291.295,291.727,292.334,291.292,295.578,292.807,291.546,291.631,292.189,291.957,0.0,0.0
4,22/09/12,11:47:12,291.294,291.728,292.334,291.292,295.578,292.807,291.546,291.630,292.189,291.957,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19977,22/09/26,09:51:42,135.366,85.338,77.491,73.340,68.620,73.626,67.923,81.292,76.005,80.287,0.0,0.0
19978,22/09/26,09:52:42,138.694,85.338,77.491,73.340,68.615,73.625,67.924,81.292,76.005,80.287,0.0,0.0
19979,22/09/26,09:53:42,138.652,85.338,77.492,73.340,68.612,73.625,67.921,81.292,76.004,80.286,0.0,0.0
19980,22/09/26,09:54:42,138.551,85.338,77.492,73.339,68.611,73.624,67.919,81.293,76.004,80.287,0.0,0.0


In [6]:
# get rid of non-numeric strings

molyblock = df["molyblock"].apply(pd.to_numeric, errors='coerce')
opt_bench = df["intermed_opt_bench"].apply(pd.to_numeric, errors='coerce')

In [33]:
df['time']

0        11:43:12
1        11:44:12
2        11:45:12
3        11:46:12
4        11:47:12
           ...   
19977    09:51:42
19978    09:52:42
19979    09:53:42
19980    09:54:42
19981    09:55:42
Name: time, Length: 19982, dtype: object

In [24]:
df['time']

0        11:43:12
1        11:44:12
2        11:45:12
3        11:46:12
4        11:47:12
           ...   
19977    09:51:42
19978    09:52:42
19979    09:53:42
19980    09:54:42
19981    09:55:42
Name: time, Length: 19982, dtype: object

In [37]:
print("|"+df['day'][0]+"|")

|22/09/12|


In [36]:
print("|" + df['time'][0] + "|")

|11:43:12|


In [41]:
df['day'] + "T" + df['time']

0        22/09/12T11:43:12
1        22/09/12T11:44:12
2        22/09/12T11:45:12
3        22/09/12T11:46:12
4        22/09/12T11:47:12
               ...        
19977    22/09/26T09:51:42
19978    22/09/26T09:52:42
19979    22/09/26T09:53:42
19980    22/09/26T09:54:42
19981    22/09/26T09:55:42
Length: 19982, dtype: object

In [46]:
# turn time strings to time objects

df["full_time"] = pd.to_datetime(df['day'] + "T" + df['time'], format="%y/%m/%dT%H:%M:%S")

In [53]:
df["full_time"]

0       2022-09-12 11:43:12
1       2022-09-12 11:44:12
2       2022-09-12 11:45:12
3       2022-09-12 11:46:12
4       2022-09-12 11:47:12
                ...        
19977   2022-09-26 09:51:42
19978   2022-09-26 09:52:42
19979   2022-09-26 09:53:42
19980   2022-09-26 09:54:42
19981   2022-09-26 09:55:42
Name: full_time, Length: 19982, dtype: datetime64[ns]

In [52]:
# convert to cumulative mins

df["delta_t"] = df["full_time"] - df["full_time"][0]
print(df["delta_t"].dt.total_seconds())

0              0.0
1             60.0
2            120.0
3            180.0
4            240.0
           ...    
19977    1202910.0
19978    1202970.0
19979    1203030.0
19980    1203090.0
19981    1203150.0
Name: delta_t, Length: 19982, dtype: float64


In [66]:
# find time and temp intervals (note one element at end will be bogus)

df["interval_t"] = np.subtract(df["delta_t"].dt.total_seconds(),np.roll(df["delta_t"].dt.total_seconds(),1))
opt_bench_dK = np.subtract(opt_bench,np.roll(opt_bench,1))
molyblock_dK = np.subtract(molyblock,np.roll(molyblock,1))

In [67]:
# find rate of change of temp [K/min]

df["bench_dKdt"] = np.divide(opt_bench_dK,df["interval_t"]) * 60
df["molyblock_dKdt"] = np.divide(molyblock_dK,df["interval_t"]) * 60

In [74]:
plt.clf()
plt.plot(df["delta_t"][10:-10],df["bench_dKdt"][10:-10])
plt.plot(df["delta_t"][10:-10],df["molyblock_dKdt"][10:-10])
plt.show()

In [65]:
opt_bench

0        292.334
1        292.334
2        292.334
3        292.334
4        292.334
          ...   
19977     77.491
19978     77.491
19979     77.492
19980     77.492
19981     77.492
Name: intermed_opt_bench, Length: 19982, dtype: float64